<a href="https://colab.research.google.com/github/youssef911-git/sectionaiplatform/blob/main/GanMnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, Reshape, Flatten, Dropout
from keras.layers import Conv2D, Conv2DTranspose, LeakyReLU
from numpy import expand_dims, ones, zeros, vstack
from numpy.random import randn, randint
from matplotlib import pyplot
# Define the discriminator model
def define_discriminator(in_shape=(28, 28, 1)):
    model = Sequential()
    # Normal
    model.add(Conv2D(64, (3, 3), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(negative_slope=0.2))
    # Downsample
    model.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(negative_slope=0.2))
    # Downsample
    model.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(negative_slope=0.2))
    # Downsample
    model.add(Conv2D(256, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(negative_slope=0.2))
    # Classifier
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    opt = Adam(learning_rate=0.02, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [2]:
# Define the generator model
def define_generator(latent_dim):
    model = Sequential()
    # Foundation for 7x7 image
    n_nodes = 128 * 7 * 7
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(negative_slope=0.2))
    model.add(Reshape((7, 7, 128)))
    # Upsample to 14x14
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(negative_slope=0.2))
    # Upsample to 28x28
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(negative_slope=0.2))
    # Output layer
    model.add(Conv2D(1, (7, 7), activation='tanh', padding='same'))
    return model

In [3]:
# Define the GAN combining generator and discriminator
def define_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    opt = Adam(learning_rate=0.02, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [4]:
# Load and prepare MNIST dataset
def load_real_samples():
    (trainX, _), (_, _) = mnist.load_data()
    # Expand to 3D, scale from [0,255] to [-1,1]
    X = expand_dims(trainX, axis=-1).astype('float32')
    X = (X - 127.5) / 127.5
    return X

In [5]:
# Select real samples
def generate_real_samples(dataset, n_samples):
    idx = randint(0, dataset.shape[0], n_samples)
    X = dataset[idx]
    y = ones((n_samples, 1))
    return X, y

In [6]:
def generate_fake_samples(generator, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    X = generator.predict(x_input)
    y = zeros((n_samples, 1))
    return X, y


In [7]:
# Generate points in latent space
def generate_latent_points(latent_dim, n_samples):
    return randn(latent_dim * n_samples).reshape(n_samples, latent_dim)

In [8]:
# Create and save a plot of generated images
def save_plot(examples, epoch, n=7):
    examples = (examples + 1) / 2.0
    for i in range(n * n):
        pyplot.subplot(n, n, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
    filename = f'generated_plot_e{epoch+1:03d}.png'
    pyplot.savefig(filename)
    pyplot.close()

In [9]:
# Evaluate performance
def summarize_performance(epoch, generator, discriminator, dataset, latent_dim, n_samples=100):
    X_real, y_real = generate_real_samples(dataset, n_samples)
    _, acc_real = discriminator.evaluate(X_real, y_real, verbose=0)
    X_fake, y_fake = generate_fake_samples(generator, latent_dim, n_samples)
    _, acc_fake = discriminator.evaluate(X_fake, y_fake, verbose=0)
    print(f'>Accuracy real: {acc_real*100:.0f}%, fake: {acc_fake*100:.0f}%')
    save_plot(X_fake, epoch)
    generator.save(f'generator_model_{epoch+1:03d}.h5')

In [11]:
# Train the GAN
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=256):
    bat_per_epo = dataset.shape[0] // n_batch
    half_batch = n_batch // 2
    for i in range(n_epochs):
        for j in range(bat_per_epo):
            X_real, y_real = generate_real_samples(dataset, half_batch)
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
            X_gan = generate_latent_points(latent_dim, n_batch)
            y_gan = ones((n_batch, 1))
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            print({i+1}, {j+1},bat_per_epo,d_loss1,d_loss2,g_loss)
        if (i+1) % 10 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)

latent_dim = 100
d_model = define_discriminator()
g_model = define_generator(latent_dim)
gan_model = define_gan(g_model, d_model)
dataset = load_real_samples()
train(g_model, d_model, gan_model, dataset, latent_dim)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 160ms/step


{1} {1} 234 0.72936165 0.7111971 [array(0.7111971, dtype=float32), array(0.7111971, dtype=float32), array(0.35546875, dtype=float32)]
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 170ms/step
{1} {2} 234 0.7183593 0.7088985 [array(0.7088985, dtype=float32), array(0.7088985, dtype=float32), array(0.38476562, dtype=float32)]
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step
{1} {3} 234 0.71370536 0.70857745 [array(0.70857745, dtype=float32), array(0.70857745, dtype=float32), array(0.39322916, dtype=float32)]
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 161ms/step
{1} {4} 234 0.7123519 0.70873547 [array(0.70873547, dtype=float32), array(0.70873547, dtype=float32), array(0.38671875, dtype=float32)]
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step
{1} {5} 234 0.7113171 0.7083787 [array(0.7083787, dtype=float32), array(0.7083787, dtype=float32), array(0.38984376, dtype=float32)]
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 263ms/step
{1} {6} 234 0.71053976 0.7080838 [array(0.7080838, dtype=float32), array(0.7080838, dtype=float32), array(0.39388022, dtype=float32)

KeyboardInterrupt: 

In [12]:
from keras.models import load_model
from numpy.random import randn
from numpy import asarray
from matplotlib import pyplot

# Generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # Generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # Reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [13]:
# Plot the generated images
def create_plot(examples, n):
    # Scale pixel values from [-1,1] to [0,1]
    examples = (examples + 1) / 2.0
    # Plot images
    for i in range(n * n):
        # Define subplot
        pyplot.subplot(n, n, 1 + i)
        # Turn off axis
        pyplot.axis('off')
        # Plot raw pixel data
        pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
    pyplot.show()


In [17]:
# Example 1: Generate a grid of random images
def generate_random_images(generator_model_path, latent_dim, grid_size):
    # Load the pre-trained generator model
    model = load_model(generator_model_path)
    # Generate latent points
    latent_points = generate_latent_points(latent_dim, grid_size * grid_size)
    # Generate images
    X = model.predict(latent_points)
    # Plot the result
    create_plot(X, grid_size)
